In [6]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings("ignore")
from stockstats import StockDataFrame

mode = "test"
if mode == "test":
    startdate="2015-01-02"
    enddate="2018-12-30"
else:
    startdate="2019-01-02"
    enddate="2019-02-28"

def get(symbols,startdate,enddate):
    def data(symbol):
        if symbol == "COM7":
            symbol = "_com7"
        data_path="../torodata/data/"
        return pd.read_csv(data_path+symbol+".csv", index_col="Date", parse_dates=True)[startdate:enddate]
    datas = map(data, symbols)
    return pd.concat(datas, keys=symbols, names=['Symbol','Date'])

def get_rsi(symbols,startdate,enddate):
    def data(symbol):
        if symbol == "COM7":
            symbol = "_com7"
        data_path="../torodata/data/"
        df = pd.read_csv(data_path+symbol+".csv", index_col="Date", parse_dates=True)[startdate:enddate]
        stock = StockDataFrame.retype(df)
        df['RSI'] = stock['rsi_2']
        return df
    datas = map(data, symbols)
    return pd.concat(datas, keys=symbols, names=['Symbol','Date'])

def get_rsi14(symbols,startdate,enddate):
    def data(symbol):
        if symbol == "COM7":
            symbol = "_com7"
        data_path="../torodata/data/"
        df = pd.read_csv(data_path+symbol+".csv", index_col="Date", parse_dates=True)[startdate:enddate]
        stock = StockDataFrame.retype(df)
        df['RSI14'] = stock['rsi_14']
        return df
    datas = map(data, symbols)
    return pd.concat(datas, keys=symbols, names=['Symbol','Date'])

symbol_file = open("set50.txt","r")
#symbol_file = open("../torodata/set100.txt","r")
symbols = [x.strip('\n') for x in symbol_file.readlines()]
#print(symbols)

def get_df_by_col_name(df, colname):
    df_before = df[[colname]].reset_index()
    df_after = df_before.pivot('Date','Symbol',colname)
    return df_after

all_data = get(symbols,startdate,enddate)
all_data_rsi = get_rsi(symbols,startdate,enddate)
all_data_rsi14 = get_rsi14(symbols,startdate,enddate)


#Get all new cols
all_data['Return']=all_data['Adj Close'].pct_change()*100
all_data['OC_Return']=(all_data['Close']-all_data['Open'])/all_data['Open']*100
all_data['Return2Day']=all_data['Adj Close'].pct_change(periods=2)*100
all_data['Return5Day']=all_data['Adj Close'].pct_change(periods=5)*100
all_data['Return10Day']=all_data['Adj Close'].pct_change(periods=10)*100
all_data['Return15Day']=all_data['Adj Close'].pct_change(periods=15)*100
all_data['Return20Day']=all_data['Adj Close'].pct_change(periods=20)*100
all_data['Return30Day']=all_data['Adj Close'].pct_change(periods=30)*100
all_data['AvgVol20']=all_data['Volume'].rolling(window=20).mean()

# All price cols
price_adj_close = get_df_by_col_name(all_data,"Adj Close")
volume = get_df_by_col_name(all_data,"Volume")
volume20 = get_df_by_col_name(all_data,"AvgVol20")
daily_return = get_df_by_col_name(all_data,"Return")
return_2_day = get_df_by_col_name(all_data,"Return2Day")
return_5_day = get_df_by_col_name(all_data,"Return5Day")
return_10_day = get_df_by_col_name(all_data,"Return10Day")
return_15_day = get_df_by_col_name(all_data,"Return15Day")
return_20_day = get_df_by_col_name(all_data,"Return20Day")
return_30_day = get_df_by_col_name(all_data,"Return30Day")
tmr_return_of_ocreturn = get_df_by_col_name(all_data,"OC_Return").shift(-1)
tmr_open_price = get_df_by_col_name(all_data,"Open").shift(-1)
all_rsi_value = get_df_by_col_name(all_data_rsi,"RSI")
all_rsi14_value = get_df_by_col_name(all_data_rsi14,"RSI14")

In [7]:
volume20

Symbol,ADVANC,AOT,BANPU,BBL,BDMS,BEM,BGRIM,BH,BJC,BPP,...,SCC,SPRC,TCAP,TISCO,TMB,TOA,TOP,TRUE,TU,WHA
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-05,NaN,6920905.0,19884510.0,24468675.0,5534195.0,NaN,NaN,3772285.0,967900.0,NaN,...,5444805.0,NaN,16914905.0,3492660.0,13859060.0,NaN,1351250.0,13503650.0,67350285.0,11552280.0
2015-01-06,NaN,7601785.0,18666640.0,24229675.0,6570870.0,NaN,NaN,3569745.0,914390.0,NaN,...,5263720.0,NaN,16504295.0,3465730.0,29285500.0,NaN,1830905.0,21249670.0,64560115.0,10816400.0
2015-01-07,NaN,8365650.0,17146565.0,21573890.0,7349445.0,NaN,NaN,3326260.0,862435.0,NaN,...,5006540.0,NaN,15506765.0,3291600.0,36125515.0,NaN,2368915.0,27506115.0,60359690.0,9583545.0
2015-01-08,NaN,9032265.0,16072985.0,20606670.0,11105380.0,NaN,NaN,3227805.0,817260.0,NaN,...,4843380.0,NaN,14831475.0,3186860.0,52581900.0,NaN,2847515.0,33635205.0,55613295.0,8962940.0
2015-01-09,NaN,10283815.0,14941825.0,19465535.0,11700310.0,NaN,NaN,3031115.0,801395.0,NaN,...,4591180.0,NaN,14273325.0,3089845.0,56732960.0,NaN,2939810.0,48601740.0,52727840.0,8462275.0
2015-01-12,NaN,10529850.0,14658820.0,19244980.0,12307475.0,NaN,NaN,2987065.0,776675.0,NaN,...,4409710.0,NaN,13830100.0,3095795.0,64105430.0,NaN,3020475.0,57168895.0,52090345.0,8009185.0
2015-01-13,NaN,10908855.0,13507070.0,17668710.0,14255000.0,NaN,NaN,2876320.0,770655.0,NaN,...,4249550.0,NaN,13149590.0,3042730.0,68636395.0,NaN,2975285.0,69834845.0,49682245.0,7950325.0
2015-01-14,NaN,11440715.0,12776690.0,17494800.0,15388805.0,NaN,NaN,2761825.0,794805.0,NaN,...,4092470.0,NaN,12658340.0,2984825.0,73452445.0,NaN,2992080.0,77442235.0,48367620.0,7922905.0
2015-01-15,NaN,11914305.0,12555380.0,16665825.0,16694575.0,NaN,NaN,2562055.0,791940.0,NaN,...,3662405.0,NaN,11671155.0,3013480.0,76258635.0,NaN,3003985.0,81316785.0,46976555.0,7328330.0


In [2]:
# LONG ONLY
total_return=0.0
for datelabel in price_adj_close.index[:-1]:
    print("-----------------------------\n")
    print("[%s]" % datelabel)
    daily_return=0.0
    # Original - SET50
    lowest_x_day_return = return_2_day.loc[datelabel].sort_values()[:10]
    lowest_x_day_return_symbols = lowest_x_day_return.index.tolist()
    # Original - SET50
    lowest_rsi = all_rsi_value.loc[datelabel].sort_values()[:30]
    lowest_rsi_symbols = lowest_rsi.index.tolist()
    for symbol in lowest_x_day_return_symbols:
        adj_close_price = price_adj_close.loc[datelabel,symbol]
        rsi = all_rsi_value.loc[datelabel,symbol]
        if pd.isnull(adj_close_price):
            print("[%s] %s does not have close price" % (datelabel, symbol))
            continue
        if rsi < 5:
            if symbol in lowest_rsi_symbols:
                next_day_return = tmr_return_of_ocreturn.loc[datelabel,symbol]
                comm_buy = tmr_open_price.loc[datelabel, symbol]*0.0015
                comm_sell = tmr_open_price.loc[datelabel, symbol]*0.0015
                total_comm = comm_buy + comm_sell
                print("BUY[%s] Close_price:%.2f, next_day_ret= %.2f, comm=%.2f" % (symbol, adj_close_price,next_day_return, total_comm))
                daily_return = daily_return + next_day_return - total_comm
                total_return = total_return + next_day_return - total_comm
    print("Daily Return: %.2f" % daily_return)
print("Total Return = %.2f percent" % total_return)

-----------------------------

[2019-01-02 00:00:00]
Daily Return: 0.00
-----------------------------

[2019-01-03 00:00:00]
BUY[SPRC] Close_price:9.40, next_day_ret= 3.72, comm=0.03
BUY[TMB] Close_price:2.18, next_day_ret= 0.92, comm=0.01
BUY[GLOBAL] Close_price:18.60, next_day_ret= -2.72, comm=0.06
BUY[HMPRO] Close_price:14.90, next_day_ret= -0.67, comm=0.04
BUY[BANPU] Close_price:14.70, next_day_ret= 6.80, comm=0.04
BUY[WHA] Close_price:4.18, next_day_ret= 0.48, comm=0.01
BUY[BJC] Close_price:47.50, next_day_ret= 1.59, comm=0.14
Daily Return: 9.79
-----------------------------

[2019-01-04 00:00:00]
BUY[GLOBAL] Close_price:17.90, next_day_ret= -2.76, comm=0.05
BUY[CENTEL] Close_price:38.75, next_day_ret= 1.94, comm=0.12
BUY[KTC] Close_price:30.50, next_day_ret= 2.42, comm=0.09
BUY[GPSC] Close_price:57.50, next_day_ret= 1.30, comm=0.17
BUY[MTC] Close_price:48.25, next_day_ret= 0.51, comm=0.15
BUY[BEM] Close_price:9.75, next_day_ret= 0.51, comm=0.03
BUY[HMPRO] Close_price:14.80, next_

In [3]:
# LONG ONLY for TODAY TRADE
datelabel="2019-02-28"
lowest_x_day_return = return_2_day.loc[datelabel].sort_values()[:10]
lowest_x_day_return_symbols = lowest_x_day_return.index.tolist()
lowest_rsi = all_rsi_value.loc[datelabel].sort_values()[:30]
lowest_rsi_symbols = lowest_rsi.index.tolist()
ok_rsi = lowest_rsi[lowest_rsi < 5]
ok_rsi_sym = lowest_rsi[lowest_rsi < 5].index.tolist()
#print(lowest_rsi)
#print(lowest_x_day_return)
#print(ok_rsi)
for sym in ok_rsi_sym:
    if sym in lowest_x_day_return_symbols:
        print("BUY: %s" % sym)

BUY: TOA
BUY: AOT
BUY: BGRIM
